In [19]:
import pandas as pd
import numpy as np
from sodapy import Socrata

In [20]:
client = Socrata("data.sfgov.org", None)
# results = client.get("cuks-n6tp", limit = 2191368)
data = client.get("cuks-n6tp", limit = 3000000)

In [21]:
data_df = pd.DataFrame.from_records(data)

In [22]:
print(type(data_df))
print(data_df.shape)

<class 'pandas.core.frame.DataFrame'>
(2192062, 13)


In [23]:
print(data_df[:4])

                          address        category                     date  \
0         2200 Block of MARKET ST  OTHER OFFENSES  2013-08-03T00:00:00.000   
1         1600 Block of HOWARD ST        WARRANTS  2013-09-29T00:00:00.000   
2    2800 Block of LEAVENWORTH ST  MISSING PERSON  2013-09-01T00:00:00.000   
3  300 Block of SOUTH VAN NESS AV         ROBBERY  2013-09-17T00:00:00.000   

  dayofweek                                          descript incidntnum  \
0  Saturday                    VIOLATION OF RESTRAINING ORDER  130648192   
1    Sunday                                    WARRANT ARREST  130822841   
2    Sunday                                      FOUND PERSON  130730824   
3   Tuesday  ROBBERY OF A COMMERCIAL ESTABLISHMENT WITH A GUN  130783798   

                                            location pddistrict  \
0  {u'type': u'Point', u'coordinates': [-122.4321...    MISSION   
1  {u'type': u'Point', u'coordinates': [-122.4172...   SOUTHERN   
2  {u'type': u'Point', u'co

In [24]:
data_df.sort_values(by=['date'])

,address,category,date,dayofweek,descript,incidntnum,location,pddistrict,pdid,resolution,time,x,y
40923,BRYANT ST / 6TH ST,WARRANTS,2003-01-01T00:00:00.000,Wednesday,ENROUTE TO OUTSIDE JURISDICTION,030000387,"{u'type': u'Point', u'coordinates': [-122.4025...",SOUTHERN,3000038762050,"ARREST, BOOKED",02:00,-122.402527594672,37.7760382838117
99950,300 Block of TOWNSEND ST,EMBEZZLEMENT,2003-01-01T00:00:00.000,Wednesday,"EMBEZZLEMENT, GRAND THEFT",031180873,"{u'type': u'Point', u'coordinates': [-122.3961...",SOUTHERN,3118087310015,"ARREST, BOOKED",12:00,-122.396164058532,37.7762328753299
13451,0 Block of PERRY ST,NON-CRIMINAL,2003-01-01T00:00:00.000,Wednesday,"DEATH REPORT, CAUSE UNKNOWN",030130188,"{u'type': u'Point', u'coordinates': [-122.3961...",SOUTHERN,3013018861030,NONE,19:12,-122.396167865037,37.7827239702919
18401,900 Block of GILMAN AV,LARCENY/THEFT,2003-01-01T00:00:00.000,Wednesday,PETTY THEFT FROM LOCKED AUTO,030005133,"{u'type': u'Point', u'coordinates': [-122.3874...",BAYVIEW,3000513306241,NONE,19:00,-122.387479514747,37.7179319466469
18392,CAPP ST / 25TH ST,ROBBERY,2003-01-01T00:00:00.000,Wednesday,ROBBERY ON THE STREET WITH A GUN,030001012,"{u'type': u'Point', u'coordinates': [-122.4172...",MISSION,3000101203011,NONE,03:57,-122.417201182665,37.7507046984612
44261,400 Block of 14TH ST,LARCENY/THEFT,2003-01-01T00:00:00.000,Wednesday,PETTY THEFT OF PROPERTY,030039596,"{u'type': u'Point', u'coordinates': [-122.4233...",MISSION,3003959606371,NONE,13:00,-122.423321421434,37.7680170125986
44256,300 Block of SERRANO DR,LARCENY/THEFT,2003-01-01T00:00:00.000,Wednesday,"LOST PROPERTY, GRAND THEFT",030003563,"{u'type': u'Point', u'coordinates': [-122.4805...",TARAVAL,3000356326145,NONE,19:30,-122.480502505559,37.7191147885897
29503,1900 Block of MARKET ST,WARRANTS,2003-01-01T00:00:00.000,Wednesday,ENROUTE TO OUTSIDE JURISDICTION,030000694,"{u'type': u'Point', u'coordinates': [-122.4256...",SOUTHERN,3000069462050,"ARREST, BOOKED",02:18,-122.425669041358,37.7702451976438
48057,100 Block of 4TH ST,ASSAULT,2003-01-01T00:00:00.000,Wednesday,AGGRAVATED ASSAULT WITH BODILY FORCE,030001498,"{u'type': u'Point', u'coordinates': [-122.4032...",SOUTHERN,3000149804014,NONE,02:45,-122.403258537277,37.7836127222201
370509,800 Block of BRYANT ST,SUSPICIOUS OCC,2003-01-01T00:00:00.000,Wednesday,SUSPICIOUS OCCURRENCE,050814180,"{u'type': u'Point', u'coordinates': [-122.4034...",SOUTHERN,5081418064070,NONE,00:01,-122.403404791479,37.775420706711


In [25]:
print(data_df.shape)

(2192062, 13)


In [26]:
data_df = data_df[data_df.date < '2016-07-01T00:00:00.000' ]

In [27]:
print(data_df[:0])

Empty DataFrame
Columns: [address, category, date, dayofweek, descript, incidntnum, location, pddistrict, pdid, resolution, time, x, y]
Index: []


In [39]:
df = data_df.drop(['incidntnum','pdid'], axis =1)
# df = df.drop(df.columns[[0]], axis =1)

In [40]:
print(df.shape)

(1937307, 11)


In [41]:
df.sort_values(by=['date'])

,address,category,date,dayofweek,descript,location,pddistrict,resolution,time,x,y
46561,1200 Block of MASON ST,LARCENY/THEFT,2003-01-01T00:00:00.000,Wednesday,GRAND THEFT FROM A BUILDING,"{u'type': u'Point', u'coordinates': [-122.4115...",CENTRAL,NONE,07:00,-122.411522842364,37.7949673860787
24384,TURK ST / TAYLOR ST,DRUG/NARCOTIC,2003-01-01T00:00:00.000,Wednesday,POSSESSION OF HEROIN,"{u'type': u'Point', u'coordinates': [-122.4107...",TENDERLOIN,"ARREST, CITED",10:10,-122.41076876635,37.7832145189783
24331,SUTTER ST / LARKIN ST,NON-CRIMINAL,2003-01-01T00:00:00.000,Wednesday,LOST PROPERTY,"{u'type': u'Point', u'coordinates': [-122.4184...",NORTHERN,NONE,13:00,-122.418467747938,37.7879652569568
33641,100 Block of TAYLOR ST,OTHER OFFENSES,2003-01-01T00:00:00.000,Wednesday,ESCAPE FROM JAIL,"{u'type': u'Point', u'coordinates': [-122.4109...",TENDERLOIN,NONE,21:30,-122.410928353347,37.7836778741124
50458,100 Block of LOUISBURG ST,WEAPON LAWS,2003-01-01T00:00:00.000,Wednesday,CARRYING A CONCEALED WEAPON,"{u'type': u'Point', u'coordinates': [-122.4496...",TARAVAL,"ARREST, BOOKED",15:24,-122.44964547844,37.7210803010507
33700,100 Block of JONES ST,WARRANTS,2003-01-01T00:00:00.000,Wednesday,ENROUTE TO OUTSIDE JURISDICTION,"{u'type': u'Point', u'coordinates': [-122.4123...",TENDERLOIN,"ARREST, BOOKED",21:41,-122.412386379783,37.7825386747303
33703,GEARY BL / 6TH AV,NON-CRIMINAL,2003-01-01T00:00:00.000,Wednesday,LOST PROPERTY,"{u'type': u'Point', u'coordinates': [-122.4642...",RICHMOND,NONE,02:00,-122.464281231868,37.7810193159964
24231,100 Block of GRIJALVA DR,LARCENY/THEFT,2003-01-01T00:00:00.000,Wednesday,PETTY THEFT OF PROPERTY,"{u'type': u'Point', u'coordinates': [-122.4796...",TARAVAL,NONE,19:00,-122.479691526653,37.715756044538
24205,1400 Block of 48TH AV,SECONDARY CODES,2003-01-01T00:00:00.000,Wednesday,PREJUDICE-BASED INCIDENT,"{u'type': u'Point', u'coordinates': [-122.5080...",TARAVAL,NONE,12:00,-122.508069752528,37.7593769103944
33762,2200 Block of MARIPOSA ST,BURGLARY,2003-01-01T00:00:00.000,Wednesday,"BURGLARY, HOT PROWL, FORCIBLE ENTRY","{u'type': u'Point', u'coordinates': [-122.4059...",MISSION,"ARREST, BOOKED",08:35,-122.405948873394,37.7633325927706


In [42]:
print(df.shape)

(1937307, 11)


In [43]:
# df.to_csv("test.csv", sep=',', encoding='utf-8')

In [44]:
print(df[:2])

                   address        category                     date dayofweek  \
0  2200 Block of MARKET ST  OTHER OFFENSES  2013-08-03T00:00:00.000  Saturday   
1  1600 Block of HOWARD ST        WARRANTS  2013-09-29T00:00:00.000    Sunday   

                         descript  \
0  VIOLATION OF RESTRAINING ORDER   
1                  WARRANT ARREST   

                                            location pddistrict  \
0  {u'type': u'Point', u'coordinates': [-122.4321...    MISSION   
1  {u'type': u'Point', u'coordinates': [-122.4172...   SOUTHERN   

       resolution   time                  x                 y  
0            NONE  19:17  -122.432198022433  37.7651107322703  
1  ARREST, BOOKED  22:00  -122.417217927213  37.7710386516388  
